In [1]:
!pip install flask
!pip install pyserial

In [12]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/')
def index():
    return 'Welcome to my capstone server'

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [10/May/2024 12:01:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/May/2024 12:01:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2024 12:01:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/May/2024 12:01:06] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
from flask import Flask, request
import os
from werkzeug.utils import secure_filename
import subprocess

app = Flask(__name__)

# 안전한 디렉토리로 변경하세요. 예: 현재 작업 디렉토리에 uploads 폴더를 사용
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# UPLOAD_FOLDER가 존재하는지 확인하고, 없으면 생성
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        save_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(save_path)
        
        # 파일 저장 후 파이썬 스크립트 실행, 시스템에 맞게 'python' 또는 'python3'을 사용하세요.
        subprocess.run(['python3', 'test1.py', save_path])
        
        return 'File uploaded and script executed'
    return 'File type not allowed'

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit
127.0.0.1 - - [04/May/2024 19:15:19] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [04/May/2024 19:15:20] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.27 - - [04/May/2024 19:15:30] "GET / HTTP/1.1" 405 -
192.168.0.27 - - [04/May/2024 19:15:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2024 19:15:42] "GET / HTTP/1.1" 405 -


In [5]:
from flask import Flask
import serial
import threading

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM3', 9600)

def read_from_port(ser):
    while True:
        reading = ser.readline().decode()
        print("Received from Arduino:", reading)

thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

@app.route('/')
def index():
    return "Serial Communication with Arduino"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit


Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!



127.0.0.1 - - [06/May/2024 18:48:31] "GET / HTTP/1.1" 200 -


Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received fro

In [10]:
from flask import Flask, request
import easyocr
import os
import subprocess
import serial

app = Flask(__name__)

# 아두이노와 통신을 위한 Serial 포트 설정
# COM 포트와 보드레이트는 프로젝트에 맞게 조정해야 합니다.
arduino = serial.Serial('COM3', 9600)

@app.route('/upload', methods=['POST'])
def save_image():
    if 'photo' in request.files:
        photo = request.files['photo']
        save_path = os.path.join('uploads', 'photo.jpg')
        photo.save(save_path)
        ocr_result = run_ocr(save_path)

        # OCR 결과를 아두이노로 보내기 전에 외부 스크립트 실행
        script_result = run_script()
        send_text_to_arduino(script_result)
        
        return ocr_result
    return 'No photo uploaded', 400

def run_ocr(image_path):
    reader = easyocr.Reader(['en','ko'])
    result = reader.readtext(image_path)
    text = ' '.join([elem[1] for elem in result])
    return text

def run_script():
    # 'main.py'는 변환된 Jupyter Notebook 스크립트의 경로입니다.
    # 실행 결과를 캡처하고 싶다면, stdout과 stderr를 subprocess.PIPE로 설정해야 합니다.
    result = subprocess.run(['python', 'C:\\Users\\peter\\capstone\\main.py'], capture_output=True, text=True)
    return result.stdout

def send_text_to_arduino(text):
    # 아두이노에 텍스트 전송
    arduino.write(text.encode())

if __name__ == '__main__':
    app.run(debug=True)


SerialException: could not open port 'COM3': PermissionError(13, '액세스가 거부되었습니다.', None, 5)

In [2]:
from flask import Flask, request, jsonify
import nbimporter
from functions import detect_objects, crop_book, text_detection
import serial
import time

app = Flask(__name__)

# COM 포트는 시스템에 따라 달라질 수 있으므로 아두이노 IDE에서 확인한 포트로 변경해야 합니다.
arduino = serial.Serial(port='COM3', baudrate=9600, timeout=.1)

def send_to_arduino(text):
    # 아두이노에 데이터를 보내는 함수
    # \n을 데이터 끝에 추가하여 아두이노가 메시지 끝을 인식하게 합니다.
    arduino.write(bytes(text + "\n", 'utf-8'))
    time.sleep(0.05) # 아두이노가 데이터를 처리할 시간을 제공합니다.
    
@app.route('/process-image', methods=['POST'])
def process_image():
    data = request.json
    
    # 요청에서 이미지 경로를 받음
    img_path = data['img_path']
    
    # 이미지에서 책을 자르는 함수 실행
    crop_book(img_path)
    
    # 자른 이미지의 경로
    img_left = "C:\\Users\\peter\\capstone\\onePage\\left_half.jpg"
    img_right = "C:\\Users\\peter\\capstone\\onePage\\right_half.jpg"
    
    # 텍스트 인식 실행
    recognized_texts = text_detection(img_left, img_right)

    # 인식된 텍스트를 아두이노로 보냄
    for text in recognized_texts:
        send_to_arduino(text)
        # 아두이노로부터 응답을 받는 부분도 추가할 수 있습니다.
        # 예: print(arduino.readline().decode('utf-8').rstrip())
    
    # 인식된 텍스트를 JSON 형태로 응답
    return jsonify(recognized_texts)

if __name__ == '__main__':
    app.run(port=5000, debug=True, host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\peter\anaconda3\envs\capstone\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
from flask import Flask
import threading
import serial
import time
import logging
logging.basicConfig(level=logging.DEBUG)

app = Flask(__name__)

# 시리얼 통신 설정
# COM 포트와 baudrate는 실제 환경에 맞게 조정해야 합니다.
ser = serial.Serial('COM3', 9600, timeout=1)

def send_hello():
    while True:
        ser.write(b'hello\n')  # 아두이노로 "hello" 문자열 전송
        time.sleep(5)  # 5초 대기

# 별도의 스레드에서 "hello" 메시지를 주기적으로 보내도록 설정
thread = threading.Thread(target=send_hello)
thread.start()

@app.route('/')
def index():
    return "Flask 서버가 실행 중입니다!"

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')


SerialException: could not open port 'COM3': PermissionError(13, '액세스가 거부되었습니다.', None, 5)

Exception in thread Thread-22 (send_hello):
Traceback (most recent call last):
  File "C:\Users\peter\anaconda3\envs\capstone\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\peter\anaconda3\envs\capstone\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\peter\AppData\Local\Temp\ipykernel_13276\214995115.py", line 16, in send_hello
  File "C:\Users\peter\anaconda3\envs\capstone\Lib\site-packages\serial\serialwin32.py", line 317, in write
    raise SerialException("WriteFile failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: WriteFile failed (PermissionError(13, '장치가 명령을 인식하지 않습니다.', None, 22))


In [21]:
from flask import Flask
import serial
import threading
import time

app = Flask(__name__)

# 시리얼 포트 설정
# COM 포트 번호와 baudrate는 환경에 맞게 조정해주세요.
ser = serial.Serial('COM3', 9600, timeout=1)

def send_hello():
    while True:
        ser.write(b'hello\n')  # 'hello' 메시지를 시리얼 포트를 통해 전송
        time.sleep(5)  # 5초 대기

@app.route('/')
def index():
    return 'Welcome to my capstone server'

if __name__ == '__main__':
    # 별도의 스레드에서 send_hello 함수를 실행하여, Flask 서버가 실행되는 동안 5초마다 'hello' 전송
    t = threading.Thread(target=send_hello)
    t.start()
    
    # Flask 애플리케이션 실행
    app.run(host='0.0.0.0')


SerialException: could not open port 'COM3': PermissionError(13, '액세스가 거부되었습니다.', None, 5)